In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
#from data_utils import get_nth_episode_info
from IPython.display import clear_output
from sapien.core import Pose
from mani_skill2.envs.pick_and_place.base_env import StationaryManipulationEnv
from mani_skill2.utils.registration import register_env
import gymnasium as gym
import sapien.core as sapien
from mani_skill2 import ASSET_DIR
from pathlib import Path
from typing import Dict, List
from mani_skill2 import format_path
from mani_skill2.utils.io_utils import load_json
import numpy as np
from sapien.core import Pose
import matplotlib.pyplot as plt
from mani_skill2.agents.base_agent import BaseAgent
import os
import warnings
# Register ManiSkill2 environments in gym
import mani_skill2.envs
from mani_skill2 import ASSET_DIR
from mani_skill2.utils.sapien_utils import look_at
from mani_skill2.utils.registration import register_env
from mani_skill2.envs.pick_and_place.pick_cube import PickCubeEnv
from mani_skill2.utils.sapien_utils import hex2rgba
from mani_skill2.sensors.camera import CameraConfig
from transforms3d.euler import euler2quat, quat2euler
from mani_skill2.utils.sapien_utils import vectorize_pose
from mani_skill2.agents.base_controller import BaseController
from IPython.display import Video
import shutil
import random
from plot_utils import *
import math

from plot_utils import *
from utils import *
from agent_env_utils import *
color_maps = {
    "red": (1, 0, 0),
    "blue": (0, 0, 1),
    "green": (0, 1, 0),
    "white": (1, 1, 1),
    "black": (0, 0, 0),
    "yellow": (1, 1, 0),
    "orange": (1, 0.5, 0),
    "purple": (0.5, 0, 0.5),
    "gray": (0.5, 0.5, 0.5),
}
directions = ["front", "behind", "right", "left", "top"]


In [2]:
%load_ext autoreload
%autoreload 2

Counting

In [11]:
data_dir = "/root/scripts/benchmark/counting"
initial_dir = f"{data_dir}/initial"
final_dir = f"{data_dir}/final"
os.makedirs(initial_dir, exist_ok=True)
os.makedirs(final_dir, exist_ok=True)
object_shapes = ["sphere", "cube"]
object_materials = ["material1", "material2"]  # Replace with actual materials if needed
object_size_range = (0.02, 0.05)
object_position_range = [(-0.5, 0.5), (-0.5, 0.5)]
num_objects_range = (1, 7)
# initialize pair of colors
strings = color_maps.keys()
pairs = list(itertools.permutations(strings, 2))
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    outputs = []
    for color_idx in trange(2):  # len(pairs)):
        color_1 = pairs[color_idx][0]
        color_2 = pairs[color_idx][1]
        for r in range(1):

            # Generate random configurations for balls and cubes
            num_ball = np.random.randint(*num_objects_range)
            num_cube = np.random.randint(*num_objects_range)
            random_ball_configs = [{
                'size': np.random.uniform(*object_size_range),
                'color': color_maps[color_1],
                'obj_type': object_shapes[0],
                'material': object_materials[0],
                'ranges': object_position_range
            } for _ in range(num_ball)]
            random_cube_configs = [{
                'size': np.random.uniform(*object_size_range),
                'color': color_maps[color_2],
                'obj_type': object_shapes[1],
                'material': object_materials[1],
                'ranges': object_position_range
            } for _ in range(num_cube)]

            configs = random_ball_configs + random_cube_configs
            configs_collate = collate_infos(configs)

            env.register_configures(configs_collate)
            _ = env.reset(options={"reconfigure": True})

            env.initialize_objects()

            save_dir_initial = f"{final_dir}/color{color_idx}_{r}_{color_1}_relative_{color_2}_ini.png"
            direction_configs=[""]
            frames = env.counting_frames("top0")
            save_images_pararell(frames, save_dir_initial)
            # collect_and_save(env, save_dir_initial, steps=1, mode="all")
            poses_initial = env.get_important_obj_poses()

            outputs.append({
                "source": save_dir_initial,
                "color": color_1,
                "num_objects": num_ball + num_cube,
                "round": r,
                "sizes": configs_collate['size'],
                "poses_initial": [list(np.float64(item)) for item in poses_initial],
            })

    write_out(f"{data_dir}/index.jsonl", outputs)

del env

[2024-08-14 05:25:25.755] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.


100%|██████████| 2/2 [00:02<00:00,  1.17s/it]


Relative Position

In [12]:
# Define directories
data_dir = "benchmark/counting/relative"
initial_dir = f"{data_dir}/initial"
final_dir = f"{data_dir}/final"
os.makedirs(initial_dir, exist_ok=True)
os.makedirs(final_dir, exist_ok=True)

# Initialize pair of colors
strings = color_maps.keys()
pairs = list(itertools.permutations(strings, 2))

# Define ranges and object parameters
object_shapes = ["sphere", "cube"]
object_materials = ["material1", "material2"]  # Replace with actual materials if needed
object_size_ranges = {
    "sphere": (0.05, 0.1),
    "cube": (0.05, 0.08),
    "random_sphere": (0.01, 0.05)
}
object_position_range = [(-0.3, 0.3), (-0.3, 0.25)]
num_random_balls = 5
directions = ["left", "right"]  # Example directions, adjust as needed

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    # Print available directions
    print(f"Available directions: {env.direction_placement_map.keys()}")

    outputs = []
    for color_idx in trange(1):  # len(pairs)):
        color_1, color_2 = pairs[color_idx]
        for r in range(1):
            cube1_config = {
                'size': np.random.uniform(*object_size_ranges["cube"]),
                'color': color_maps[color_1],
                'obj_type': "cube",
                'ranges': object_position_range
            }
            cube2_config = {
                'size': np.random.uniform(*object_size_ranges["cube"]),
                'color': color_maps[color_2],
                'obj_type': "cube",
                'ranges': object_position_range
            }
            sphere1_config = {
                'size': np.random.uniform(*object_size_ranges["sphere"]),
                'color': color_maps[color_1],
                'obj_type': "sphere",
                'ranges': object_position_range
            }
            random_ball_configs = [{
                'size': np.random.uniform(*object_size_ranges["random_sphere"]),
                'color': color_maps["purple"],
                'obj_type': "sphere",
                'ranges': [(-0.5, 0.3), (-0.5, 0.4)]
            } for _ in range(num_random_balls)]

            configs = [sphere1_config, cube2_config] + random_ball_configs
            configs_collate = collate_infos(configs)

            env.register_configures(configs_collate)
            _ = env.reset(options={"reconfigure": True})

            env.initialize_objects()
            obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
            save_obs(obs, f"{initial_dir}/color{color_idx}_{r}.png", mode="all")
            initilization_poses = env.get_important_obj_poses(mode="all")

            for i, direction in enumerate(directions):
                if direction not in env.direction_placement_map:
                    print(f"Skipping invalid direction: {direction}")
                    continue

                env.set_poses(initilization_poses)
                direction_configs = [(0, 1, direction)]
                env.place_cubes_in_direction(direction_configs)

                save_dir_initial = f"{final_dir}/color{color_idx}_{r}_{color_1}_{direction}_{color_2}_ini.png"
                frames = env.counting_frames("top0")
                save_images_pararell(frames, save_dir_initial)  
                # collect_and_save(env, save_dir_initial, steps=1, mode="all")
                poses_initial = env.get_important_obj_poses()

                save_dir_final = f"{final_dir}/color{color_idx}_{r}_{color_1}_{direction}_{color_2}.png"
                frames = env.counting_frames("top0")
                save_images_pararell(frames, save_dir_final)
                # collect_and_save(env, save_dir_final, steps=10, mode="all")

                poses_final = env.get_important_obj_poses()

                bad_case = any(map(lambda x: check_move(x[0], x[1]), zip(poses_initial, poses_final)))
                if bad_case:
                    print(f"Bad initialization on the {direction} direction, in {color_idx} iter")
                    print(f"Obj1 pose initial {poses_initial[0]}")
                    print(f"Obj2 pose initial {poses_initial[1]}")

                outputs.append({
                    "source": save_dir_final,
                    "moving_obj": color_1,
                    "direction": direction,
                    "ref_obj": color_2,
                    "bad_case": bad_case,
                    "round": r,
                    "sizes": configs_collate['size'],
                    "poses_final": [list(np.float64(item)) for item in poses_final],
                })

    write_out(f"{data_dir}/index.jsonl", outputs)

del env

[2024-08-14 05:40:40.823] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.


Available directions: dict_keys(['behind', 'front', 'left', 'right', 'top'])


100%|██████████| 1/1 [00:05<00:00,  5.15s/it]


Spatial Occupancy

In [13]:
def generate_volume_comparison(env, obj_infos, save_info):
    outputs = []
    configs = []
    episode_id = save_info['episode_id']
    final_dir = save_info['output_dir']

    obj_names = obj_infos['obj_name']
    size_settings = obj_infos['size_settings']
    initiate_area = obj_infos['initiate_area']
    bound_directions = obj_infos['bound_direction']
    static = obj_infos['static']
    if "top" in bound_directions:
        static = True
    sizes = [round(np.random.uniform(num * 0.02, num * 0.02 + 0.01, 1).tolist()[0], 5) for num in size_settings]
    colors = obj_infos['color']
    color_ids = [color_maps[color] for color in colors]
    base_size = 0.08

    bounds = []
    for i, obj_name in enumerate(obj_names):
        obj_type = obj_name.split("-")[0]
        obj_id = obj_name.split("-")[-1]
        actual_size = sizes[i] * 1.1 / 2 if obj_type == "custom" else sizes[i]

        if obj_id == "bound":
            bounds.append(i)

        configs.append({
            'obj_type': obj_type,
            'name': obj_id,
            'size': actual_size,
            'color': color_ids[i],
            'ranges': initiate_area,
            'static': static
        })

    extra_configs = [configs[bound_id].copy() for bound_id in bounds for _ in range(7)]
    env.register_configures(collate_infos(configs))
    if extra_configs:
        env.register_configures(collate_infos(extra_configs), replace=False)
    _ = env.reset(options={"reconfigure": True})

    env.initialize_objects(regular=True)

    main_placements = [(i + 1, i, "right") for i in range(len(configs) - 1)]

    distances = [np.random.uniform(3 * base_size, 3.5 * base_size, size=1)[0] * np.sqrt(2)]
    for direction_config in main_placements:
        if direction_config[2] == "top" and obj_type == "custom":
            env.place_cubes_in_direction(direction_config, distances=distances + 2 * base_size)
        else:
            env.place_cubes_in_direction(direction_config, distances=distances)

    for k, bound_id in enumerate(bounds):
        id_offset = k * 7 + len(configs)
        if bound_directions[k] == "front":
            bound_direction_configs = [
                (0 + id_offset, bound_id, "left"), (bound_id, bound_id, "right"),
                (1 + id_offset, 0 + id_offset, "behind"), (2 + id_offset, 1 + id_offset, "right"),
                (3 + id_offset, 2 + id_offset, "right"), (4 + id_offset, 0 + id_offset, "front"),
                (5 + id_offset, 4 + id_offset, "right"), (6 + id_offset, 5 + id_offset, "right")
            ]
            env.place_cubes_in_direction(bound_direction_configs, distances=[2 * configs[bound_id]['size']] * len(bound_direction_configs))

        elif bound_directions[k] == "top":
            bound_direction_configs = [
                (0 + id_offset, bound_id, "left"), (1 + id_offset, bound_id, "right"),
                (2 + id_offset, 0 + id_offset, "top"), (3 + id_offset, 1 + id_offset, "top"),
                (4 + id_offset, 2 + id_offset, "top"), (5 + id_offset, 3 + id_offset, "top"),
                (6 + id_offset, 5 + id_offset, "left")
            ]
            dist_list = [2.5 * configs[bound_id]['size'] if cfg[2] == "top" else 2 * configs[bound_id]['size'] for cfg in bound_direction_configs]
            env.place_cubes_in_direction(bound_direction_configs, distances=dist_list)

    save_dir_initial = f"{final_dir}/volume{episode_id}.png"
    frames = env.counting_frames("top0")
    save_images_pararell(frames, save_dir_initial)
    # collect_and_save(env, save_dir_initial, steps=5, mode="all")

    outputs.append({
        "source": save_dir_initial,
        "obj_names": obj_names,
        'sizes': sizes,
        "colors": colors,
        "initiate_area": initiate_area,
        "bound_directions": bound_directions
    })
    return outputs

def generate_fitness(env, obj_infos, save_info, same_obj=True):
    outputs = []
    configs = []
    episode_id = save_info['episode_id']
    final_dir = save_info['output_dir']

    obj_names = obj_infos['obj_name']
    size_settings = obj_infos['size_settings']
    sizes = [round(np.random.uniform(num * 0.02, num * 0.02 + 0.01, 1).tolist()[0], 5) for num in size_settings]
    colors = obj_infos['color']
    color_ids = [color_maps[color] for color in colors]
    base_size = 0.08
    num_obj_mid = 2

    bounds = []
    for i, obj_name in enumerate(obj_names):
        obj_type = obj_name.split("-")[0]
        obj_id = obj_name.split("-")[-1]
        actual_size = sizes[i] * 1.1 / 2 if obj_type == "custom" else sizes[i]

        if obj_id == "bound":
            bounds.append(i)

        configs.append({
            'obj_type': obj_type,
            'name': obj_id,
            'size': actual_size,
            'color': color_ids[i],
            'ranges': [(-0.1, 0.25), (-0.3, -0.2)],
            'static': False
        })

    if same_obj:
        configs[1] = configs[0].copy()

    extra_configs = [configs[bound_id].copy() for bound_id in bounds for _ in range(2 * (num_obj_mid + 2) + 1)]
    env.register_configures(collate_infos(configs))
    if extra_configs:
        env.register_configures(collate_infos(extra_configs), replace=False)
    _ = env.reset(options={"reconfigure": True})

    env.initialize_objects(regular=True)

    main_placements = [(i + 1, i, "right") for i in range(len(configs) - 1)]

    distances = [np.random.uniform(4 * base_size, 5 * base_size, size=1)[0] * np.sqrt(2)]
    for direction_config in main_placements:
        if direction_config[2] == "top" and obj_type == "custom":
            env.place_cubes_in_direction(direction_config, distances=distances + 2 * base_size)
        else:
            env.place_cubes_in_direction(direction_config, distances=distances)

    for k, bound_id in enumerate(bounds):
        id_offset = k * (2 * (num_obj_mid + 1) + 1) + len(configs)
        bound_direction_configs = [(0 + id_offset, bound_id, "left")]
        for i in range(num_obj_mid):
            bound_direction_configs.append((bound_id, bound_id, "right"))
        for line in [1, 2]:
            bound_direction_configs.append((line + id_offset, 0 + id_offset, "behind") if line == 1 else (line + id_offset, 0 + id_offset, "front"))
            bound_direction_configs += [(line + (i + 1) * 2 + id_offset, line + i * 2 + id_offset, "right") for i in range(num_obj_mid + 1)]
        env.place_cubes_in_direction(bound_direction_configs, distances=[2 * configs[bound_id]['size']] * len(bound_direction_configs))

        shrink_configs = [(i * 2 + 1 + id_offset, i * 2 + 1 + id_offset, "front") for i in range(num_obj_mid)]
        shrink_configs += [(i * 2 + 2 + id_offset, i * 2 + 2 + id_offset, "behind") for i in range(num_obj_mid)]
        env.place_cubes_in_direction(shrink_configs, distances=[configs[bound_id]['size'] / 6 for _ in range(len(shrink_configs))])

    save_dir_initial = f"{final_dir}/fitness{episode_id}_round{r}.png"
    frames = env.counting_frames("top0")
    save_images_pararell(frames, save_dir_initial)
    #collect_and_save(env, save_dir_initial, steps=5, mode="all")

    source_pths = [save_dir_initial.replace(".png", f"_{postfix}.png") for postfix in ["front", "side", "top"]]

    answer = "yes" if size_settings[0] > size_settings[-1] else "no"

    outputs.append({
        "source": source_pths,
        "obj_names": obj_names,
        'sizes': sizes,
        "colors": colors,
        "round": r,
        "answer": answer
    })
    return outputs

In [15]:
# Define directories
data_dir = "/root/scripts/benchmark/occupancy/volume_full"
initial_dir = f"{data_dir}/initial"
final_dir = f"{data_dir}/final"
os.makedirs(initial_dir, exist_ok=True)
os.makedirs(final_dir, exist_ok=True)

flush = True
if flush:
    if os.path.exists(final_dir):
        shutil.rmtree(final_dir)
        os.makedirs(final_dir)
    if os.path.exists(os.path.join(data_dir, "index.jsonl")):
        os.remove(os.path.join(data_dir, "index.jsonl"))

# Define color pairs and other parameters
strings = color_maps.keys()
pairs = list(itertools.permutations(strings, 7))
random.seed(42)
random.shuffle(pairs)

size_types = [1, 3]
initiate_areas = [
    [(0.15, 0.25), (-0.5, -0.4)],
    [(-0.25, -0.15), (-0.5, -0.4)],
    [(0.15, 0.25), (0.1, 0.2)],
    [(-0.25, -0.15), (0.1, 0.2)]
]
space_types = ["cube-bound", "cube", "sphere", "sphere-bound"]
bound_types = ["front", "top"]

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    outputs = []
    idx = 0
    for size_settings in itertools.permutations(size_types, len(size_types)):
        for obj_names in itertools.permutations(space_types, 2):
            for colors in pairs[:7]:
                for initiate_area in initiate_areas:
                    num_bound_cases = sum([1 if "bound" in obj else 0 for obj in obj_names])
                    for bound_directions in itertools.product(bound_types, repeat=num_bound_cases):
                        obj_infos = {
                            'obj_name': obj_names,
                            'size_settings': size_settings,
                            'color': colors,
                            'initiate_area': initiate_area,
                            'bound_direction': bound_directions,
                            'static': True
                        }

                        save_infos = {
                            'episode_id': idx,
                            'output_dir': final_dir,
                        }
                        outputs += generate_volume_comparison(env, obj_infos, save_infos)
                        idx += 1

    write_out(f"{data_dir}/index.jsonl", outputs)

del env

[2024-08-14 05:48:10.079] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.


KeyboardInterrupt: 